In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
from collections import Counter
from matplotlib import cm
from matplotlib.colors import ListedColormap

import io
from pysam import VariantFile
from matplotlib_venn import venn3
import venn

In [ ]:
tab20 = cm.get_cmap('tab10', 8)
newcmap_list = []
for i in range(4):
    newcmap_list.append(tab20.colors[i])
for i in range(4):
    newcmap_list.append(np.array(list(tab20.colors[i][:-1]) +[0.3]))

newcmap = ListedColormap(newcmap_list, name='newcmap')

In [ ]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    res = pd.read_csv(
        io.StringIO(''.join(lines[1:])),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})
    return res

# VCF file

In [ ]:
vcf_pd = read_vcf("../data/2015-07-31_PATIENTID/PATIENTID-ensemble-annotated.vcf")


vcf_pd['callers'] = vcf_pd['INFO'].apply(foo)
vcf_pd['type'] = np.nan
vcf_pd['type'][vcf_pd['ALT'].str.len() - vcf_pd['REF'].str.len() == 0] = 'SNV'
vcf_pd['type'][vcf_pd['ALT'].str.len() - vcf_pd['REF'].str.len() > 0] = 'INS'
vcf_pd['type'][vcf_pd['ALT'].str.len() - vcf_pd['REF'].str.len() < 0] = 'DEL'
vcf_pd['type'][vcf_pd['ID'].str.contains('rs')] = 'SNP'
vcf_pd['freebayes'] = vcf_pd['INFO'].str.contains('freebayes')
vcf_pd['vardict'] = vcf_pd['INFO'].str.contains('vardict')
vcf_pd['varscan'] = vcf_pd['INFO'].str.contains('varscan')

vcf_pd[vcf_pd['type'] == 'INS']

In [ ]:
vcf_pd.groupby('CHROM').size()

# Bam File

# Filter Bam w.r.t. VCF file

In [ ]:
samfile = pysam.AlignmentFile("../data/healthy_chr22_merged-ready.bam", "rb")

# ititiate list of reads to remove
reads2remove = []

# iterate over positions
for count, mutation in vcf_pd[vcf_pd['type'] == 'INS'].iterrows():
    print('#####')
    print(count)
#mutation = vcf_pd.iloc[94198]
    #print(mutation)

    c = 0
    t = 0

    # get reads in this position
    if mutation['type'] == 'INS':
        for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']):
            # get supporting reads
            # print(read) 
            seq = read.query_alignment_sequence
            pos = (mutation['POS'] -1) - read.reference_start + 1 
            #print(mutation['POS'], read.reference_start)
            #print(pos)
            #print(seq[: pos+len(mutation['ALT'])], seq[pos-1:  pos-1+len(mutation['ALT'])],mutation['ALT'])
            #print(read.query_alignment_length, len(seq), read.cigarstring)
            cigar = read.cigarstring
            #print(cigar)
            if cigar is not None:
                cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                cigar_states = re.split('[0-9]+',cigar)[1:]
                cond2 = False
                if 'I' in cigar_states:
                    cumul = 0
                    indel_pos = None
                    for i, cp in enumerate(cigar_pos):
                        if (cp != 'S') and (cumul <= pos):
                            cumul += -int(cp) if cp == 'D' else int(cp)
                            indel_pos = i
                    #print(indel_pos)
                    #print(cigar_pos, indel_pos)
                    if cigar_states[indel_pos] == 'I':
                        #print(cigar_states[indel_pos])
                        cond2 = True
                #print(cigar_states, cigar_pos)
                #print(cond2)
                if (seq[pos-1:  pos-1+len(mutation['ALT'])] == mutation['ALT']) and cond2:
                    print(read.query_name) 
                reads2remove.append(read.query_name)
    elif (mutation['type'] == 'SNV') or (mutation['type'] == 'SNP'):
        for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']):
            t += 1
            # print('yes')
            # get supporting reads
            # print(read)
            seq = read.query_alignment_sequence
            # print(seq)
            pos = (mutation['POS'] -1) - read.reference_start + 1
            #print(seq[: pos], seq[pos-1], mutation['ALT'])
            if len(seq) <= pos-1:
                print('length issue')
            else:
                #print(mutation['ALT'], mutation['REF'])
                #print(read.cigarstring)
                if (seq[pos-1] == mutation['ALT']):
                    c += 1
                    #print(read.query_name) 
                    reads2remove.append(read.query_name)
        print(100*c/t)
    elif mutation['type'] == 'DEL':
        for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']):
        # get supporting reads
            #print(read) 
            #print(read.cigarstring)
            seq = read.query_alignment_sequence
            #print(seq)
            pos = (mutation['POS'] -1) - read.reference_start + 1 
            #print(pos)
            #print(seq[: pos-1+len(mutation['REF'])], seq[pos-1:  pos-1+len(mutation['REF'])], mutation['REF'], mutation['ALT'])
            #print(read.query_alignment_length, len(seq), read.cigarstring)
            cigar = read.cigarstring
            if cigar is None:
                print('cigar is None')
            else:
                #print(cigar)
                cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                cigar_states = re.split('[0-9]+',cigar)[1:]
                cond2 = False
                if 'D' in cigar_states:
                    cumul = 0
                    indel_pos = None
                    for i, cp in enumerate(cigar_pos):
                        if (cp != 'S') and (cumul <= pos):
                            cumul += -int(cp) if cp == 'D' else int(cp)
                            indel_pos = i
                    #print(indel_pos)
                    if cigar_states[indel_pos] == 'D':
                        #print(cigar_states[indel_pos])
                        cond2 = True
                #print(cigar_states, cigar_pos)
                #print(cond2)
                if (seq[pos-1:  pos-1+len(mutation['REF'])] != mutation['REF']) and cond2:
                    print(read.query_name) 
                    reads2remove.append(read.query_name)


samfile.close()

# filter bam with list

In [ ]:
samfile = pysam.AlignmentFile("../data/healthy_chr22_merged-ready.bam", "rb")

# ititiate list of reads to remove
reads2remove = []

# iterate over positions
for _, mutation in vcf_pd.iterrows():

    c = 0
    t = 0

    if mutation['type'] == 'INS':
        for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']): 
            t += 1
            seq = read.query_alignment_sequence
            pos = (mutation['POS'] -1) - read.reference_start + 1 
            cond1 = (seq[pos-1:  pos-1+len(mutation['ALT'])] == mutation['ALT'])
            cigar = read.cigarstring
            if cigar is None:
                print('INS cigar is None')
            else:
                cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                cigar_states = re.split('[0-9]+',cigar)[1:]
                cond2 = False
                if 'I' in cigar_states:
                    cumul = 0
                    indel_pos = None
                    for i, cp in enumerate(cigar_pos):
                        if (cp != 'S') and (cumul <= pos):
                            cumul += -int(cp) if cp == 'D' else int(cp)
                            indel_pos = i
                    if cigar_states[indel_pos] == 'I':
                        cond2 = True
                if cond1 and cond2:
                    c += 1
                    reads2remove.append(read.query_name)
        print(mutation['type'], 100*c/t, t)

    elif (mutation['type'] == 'SNV') or (mutation['type'] == 'SNP'):
        for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']):
            t += 1
            seq = read.query_alignment_sequence
            pos = (mutation['POS'] -1) - read.reference_start + 1
            if len(seq) <= pos-1:
                pass
                #print('length issue')
            else:
                if (seq[pos-1] == mutation['ALT']):
                    c += 1
                    reads2remove.append(read.query_name)
        #print(mutation['type'], 100*c/t, t)

    elif mutation['type'] == 'DEL':
        for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']):
            t += 1
            seq = read.query_alignment_sequence
            pos = (mutation['POS'] -1) - read.reference_start + 1 
            cond1 = (seq[pos-1:  pos-1+len(mutation['REF'])] != mutation['REF'])
            cigar = read.cigarstring
            if cigar is None:
                print('cigar is None')
            else:
                cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                cigar_states = re.split('[0-9]+',cigar)[1:]
                cond2 = False
                if 'D' in cigar_states:
                    cumul = 0
                    indel_pos = None
                    for i, cp in enumerate(cigar_pos):
                        if (cp != 'S') and (cumul <= pos):
                            cumul += -int(cp) if cp == 'D' else int(cp)
                            indel_pos = i
                    if cigar_states[indel_pos] == 'D':
                        cond2 = True
                if cond1 and cond2:
                    c += 1
                    reads2remove.append(read.query_name)
        print(mutation['type'], 100*c/t, t)

samfile.close()

In [ ]:
print(vcf_pd.shape)

In [ ]:
samfile = pysam.AlignmentFile("../data/healthy_chr22_merged-ready.bam", "rb")

# ititiate list of reads to remove
reads2remove = []

# iterate over positions
for n, mutation in vcf_pd.iterrows():
    print('###############')
    print(n)
    print(mutation['type'])

    c = 0
    t = 0

    if mutation['type'] == 'INS': # INSERTION
        for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']):
            t += 1
            seq = read.query_alignment_sequence
            pos = mutation['POS'] - read.reference_start
            cond1 = seq[pos-1:  pos-1+len(mutation['ALT'])] == mutation['ALT']
            cond2 = False
            cigar = read.cigarstring
            cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
            cigar_states = re.split('[0-9]+',cigar)[1:]
            if 'I' in cigar_states:
                cumul = 0
                indel_pos = None
                for i, cp in enumerate(cigar_pos):
                    if (cigar_states[i] != 'S') and (cumul <= pos):
                        cumul += int(cigar_pos[i])
                        indel_pos = i
                if indel_pos is not None:
                    if cigar_states[indel_pos] == 'I':
                        cond2 = True
            if cond1 and cond2:
                c += 1 
                reads2remove.append(read.query_name)
        print(100*c/t, t)
    elif (mutation['type'] == 'SNV') or (mutation['type'] == 'SNP'): # SNP/SNV
        for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']):
            t += 1
            seq = read.query_sequence
            pos = mutation['POS'] - read.reference_start + 1
            if len(seq) >= pos:
                if (seq[pos-1] == mutation['ALT']):
                    c += 1
                    reads2remove.append(read.query_name)
            else:
                print(len(seq), pos)
        print(100*c/t, t)
    elif mutation['type'] == 'DEL': # DELETION
        for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']):
            t += 1
            seq = read.query_alignment_sequence
            pos = mutation['POS'] - read.reference_start
            cond1 = (seq[pos-1:  pos-1+len(mutation['REF'])] != mutation['REF'])
            cigar = read.cigarstring
            if cigar is not None:
                cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                cigar_states = re.split('[0-9]+',cigar)[1:]
                cond2 = False
                if 'D' in cigar_states:
                    cumul = 0
                    indel_pos = None
                    for i, cp in enumerate(cigar_pos):
                        if (cp != 'S') and (cumul <= pos):
                            cumul += -int(cp) if cp == 'D' else int(cp)
                            indel_pos = i
                    if indel_pos is not None:
                        if cigar_states[indel_pos] == 'D':
                            cond2 = True
                if cond1 and cond2:
                    c += 1
                    reads2remove.append(read.query_name)
        print(100*c/t, t)

    
samfile.close()

# filter bam with list

In [ ]:

    for read in samfile.fetch(mutation['CHROM'], mutation['POS'], mutation['POS']+len(mutation['REF'])):
        # get supporting reads
        print(read)
        seq = read.query_alignment_sequence
        print(seq)
        if (seq[mutation['POS']:  mutation['POS']+len(mutation['REF'])] != mutation['REF']) and (read.query_alignment_length - len(get_forward_sequence) > 0):
            print(read.query_name) 
            reads2remove.append(read.query_name)

In [ ]:
a = 'GCAGCCACTCAGGATGTTGGAACCTGGCCATCCCTGCTTCTTTCAGTGGGTGAGGTTGGTGGCTGCTCCACCTGTTCCAGGCACACCCTTAACAGAGGTGGCTGCTTGCTCTTTAAGCCAGCTTGGCCTTGCCTGGCATGCACAGGCCCCG'
b = 'GCAGCCACTCAGGATGTTGGAACCTGGCCATCCCTGCTTCTTTCAGTGGGTGAGGTTGGTGGCTGCTCCACCTGTTCCAGGCACACCCTTAACAGAGGTGGCTGCTTGCTCTTTAAGCCAGCTTGGCCTTGCCTGGCATGCACAGGCCCCG'

a == b